# MOD_SIM_P_3 Workspace

### By Corey Cochran-Lepiz,
###   Shawn Albertson
   
Orbital Trajectories

### First step in any good code:import your libraries!


In [2]:
from math import *
from modsim import *

### Setup your units

In [3]:
m = UNITS.meter
km = UNITS.kilometer
s = UNITS.second
kg = UNITS.kilogram
N = kg * m/(s**2)
kN = kg * m/(s**2) * 1000
G = 6.67408 * (10**-11) * m**3 * kg**-1 * s**-2


1.96e+26

In [4]:
sun = System(x = 0 * m,
             y = 0 * m,
             mass = 1.9891e30 * kg             
            )

mars = System(x= -227.9e9 * m,
              y= 0 * m ,
              radius = 3396200 * m,
              mass = .64171e24 * kg    
                )

saturn = System(x = -1.429e12 * m,
                y = 0 * m,
                mass = 5.683e26 * kg)

rocket = System(x= -1 * mars.radius,
                y= 0 * m,
                dry_mass = 1433788 * kg,
                rho = 0.02 * kg/m**3, #needs function to update
                h = 0 * m, #height
                thrust = 22800000 * N,
                fuel_init = 1000000 * kg
               )


## Things to make for next time
### A function that calculates:
### Height from planet, gravity (with vector pointing to planet), mass (as we lose propellant), thrust (stages? 3).

In [6]:
def calc_mass_r(t):
    rocket.fuel_init * -.18 * t
    return rocket.dry_mass + rocket.fuel_init
    
calc_mass_r(30)

<Quantity(2433788, 'kilogram')>

In [73]:
def calc_mgrav(vr,vm, mass_rocket):
    """
    Given the vectors of two objects in space, find the force of gravity acting upon them.
    Pass a vector with the x and y positons of the rocket and body in question
    G is the gravitational constant
    vr is the vector of the rocket
    vm is the vector of mars
    height is just the distance between the center of the two objects
    returns force vector from the rocket towards the body 
    """
    height = vr.dist(vm)
    grav = G * mars.mass * mass_rocket / (height)**2
    print(height,grav/mass_rocket)
    
    a = Vector(vm.x-vr.x,vm.y-vr.y) #I found the angle to the object from the rocket
    x,y = pol2cart(a.angle,grav) #turned the angle and grav into cartesian
    
    f_grav = Vector(x,y)
    return f_grav

def calc_satgrav(vr,vsat,mass_rocket):
    height = vr.dist(vsat)
    grav = G * saturn.mass * mass_rocket / (height)**2

    a = Vector(vsat.x-vr.x,vsat.y-vr.y)
    x,y = pol2cart(a.angle,grav)
    
    f_grav = Vector(x,y)
    return f_grav

def calc_sgrav(vr,mass):
    height = vr.dist(Vector(0,0)*m)
    grav = G * sun.mass * mass / (height)**2
    
    x,y = pol2cart(vr.angle,grav)
    
    f_grav = Vector(x,y)
    return f_grav
    
#TROUBLESHOOTING
    
#%time calc_sgrav(1.429e12,rocket.dry_mass+rocket.fuel_init)/(rocket.dry_mass+rocket.fuel_init)
vr = Vector(1,1) * m
vm = Vector(-1,1) * m
#%time sqrt((vr.x-vm.x)**2+(vr.y-vm.y)**2)
#%time (vr.diff_angle(vm)).to(UNITS.degree)
vr.mag
#%time acos((vr.x*vm.x+vr.y*vm.y)/sqrt((vr.x**2+vr.x**2)*(vm.x**2+vm.y**2)))
a = Vector(vm.x-vr.x,vm.y-vr.y)
x,y = pol2cart(a.angle,1)
c = Vector(x,y)
#vr.dist(vm)
#print(c.angle,c.mag)

b = calc_mgrav(Vector(-227.9e9*m - mars.radius,0*m),Vector(-227.9e9*m,0*m),rocket.dry_mass+rocket.fuel_init)
b.mag/(rocket.dry_mass+rocket.fuel_init)
b.angle
print(b.mag)

3396200.0 meter 3.7131603124948063 meter / second ** 2
9037045.01062611 kilogram * meter / second ** 2


In [74]:
#a = calc_mass_r(100)
#calc_grav(3000, a)

In [10]:
def height_from_mars(x,y): #Assuming that mars is at the origin
    """"
    Given an x and y returns the distance from the center
    of Mars assuming that Mars is at the origin.
    
    """"
    #[theta,rho] = cart2pol(x,y) #but it doens't like big numbers
    x /= m #strip the units
    y /= m
    rho = sqrt(x**2+y**2) * m #distance formula
    height = rho - (3396200*m)
    return height

def force_of_drag(v,h):
    """"
    Returns the force of drag given velocity of the object
    and it's distance away from the planet
    
    rho is the air density
    v is velocity of rocket
    c_d is drag coeffeciant of drag of the rocket but it's dependant on velocity
    """"
    C_d = drag(v)
    return -(1/2) * rho * v.mag * v * C_d * area

def rho(h): #math from NASA https://www.grc.nasa.gov/www/k-12/airplane/atmosmrm.html
    """"
    Returns the air density of Mars above the ground given the height
    """"
    if h < 7000: #two zones for density
        T = -31 - 0.000998 * h #temp
        P = .699*exp(-0.00009*h) #pressure
    else:
        T = -23.4 - 0.00222*h
        P = .699*exp(-0.00009*h)
    return P/(.1921*(T+273.1)) #return density

def accel_thrust(system,v,t): #UNDONE
    """"
    Returns an accel vector of thrust given a system, the objects velocity, and time.
    """"
    unpack(system)
    
    
    return a_thrust

In [ ]:
def slope_func(state,t,system):
    x, y, vx, vy = state
    unpack(system)
    
    h = height_from_mars(x,y)
    a_grav = calc_grav(h)
    
    v = Vector(vx,vy)
    
    f_drag = force_of_drag(v.mag,h)
    a_drag = d_drag/mass
    
    a_thrust = accel_thrust(system,v,t)
    
    a = a_grav + a_drag + a_thrust
    
    return v.x, v.y, a.x, a.y